In [ ]:
# !pip3 install chardet -i https://mirrors.aliyun.com/pypi/simple/

In [ ]:
import glob
import os
from tqdm import tqdm
import numpy as np
import shutil

In [ ]:
character_dir = {
    "Openness":  {
        "words": "intellectual, imaginative, independent-minded",
        "opposite": "Closeness",
        "explanation": "...",
    },
    "Conscientiousness":  {
        "words": "orderly, responsible, dependable",
        "opposite": "Lack of direction",
        "explanation": "...",
    },
    "Extraversion":  {
        "words": "talkative, assertive, energetic",
        "opposite": "Introversion",
        "explanation": "...",
    },
    "Agreeableness":  {
        "words": "good-natured, cooperative, trustful",
        "opposite": "Antagonism",
        "explanation": "...",
    },
    "Neuroticism":  {
        "words": "emotioonal instability, irritability, anxiety, self-doubt, depression",
        "opposite": "Emotion stability",
        "explanation": "...",
    },
}
conversation_traits = {
    "Number of Turns": "The number of turns, defined as text responses separated by 700 ms, excluding backchannels and unsuccessful interjections",
    "Talking Time per Turn": "The average duration of speech per turn",
    "Frequency of Laughter": "The number of laughter occurrences per minute of speech",
    "Frequency of Emotive Backchannel": "The number of emotive backchannels per minute during another speaker's speech",
    "Frequency of Cognitive Backchannel": "The number of cognitive backchannels per minute during another speaker's speech",
    "Frequency of Interjections": "The total count of successful and unsuccessful interjections",
    "Positive Sentiment": "Tendency to say something positive",
    "Neutral Sentiment": "Tendency to say something neutral sentiment",
    "Negative Sentiment": "Tendency to say something negative",
    "Emotional Statement (Anger)": "Tendency to say something with angry emotion",
    "Emotional Statement (Disgust)": "Tendency to say something with disgust emotion",
    "Emotional Statement (Fear)": "Tendency to say something with fear emotion",
    "Emotional Statement (Joy)": "Tendency to say something with joy emotion",
    "Emotional Statement (Neutral)": "Tendency to say something with neutral emotion",
    "Emotional Statement (Sadness)": "Tendency to say something with sad emotion",
    "Emotional Statement (Surprise)": "Tendency to say something with surprise emotion",
}

# MUSHRA

In [ ]:
    # "Text": "\\nTarget Personality: '{cha}'\\nWords: {character_dir[cha]['words']}\\nExplanation: {character_dir[cha]['explanation']}",

In [ ]:
save = True
testname = "Expected Trend of Conversation Traits"
pngfile = "img/alignment.png"
def init():
    whole_text = f"""var TestConfig = {{
      "TestName": "{testname}",
      "RateScalePng": "{pngfile}",
      "RateScaleBgPng": "img/scale_abs_background.png",
      "RateMinValue": 0,
      "RateMaxValue": 100,
      "RateDefaultValue":0,
      "ShowFileIDs": false,
      "ShowResults": false,
      "LoopByDefault": false,
      "EnableABLoop": true,
      "EnableOnlineSubmission": false,
      "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
      "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
      "RandomizeTestOrder": true,
      "MaxTestsPerRun": 15,
      "RequireMaxRating": false,
      "AudioRoot": "",
      "Testsets":
    ["""
    return whole_text
whole_text = init()
for cha in character_dir:
    a = "opposite"
    text = f"<b>Target Personality</b>: {cha}<br><b>Explanation to '{cha}'</b>: {character_dir[cha]['words']}<br><b>Opposite Term</b>: '{character_dir[cha][a]}'"
    # text += "<br><br><br>"
    # for trait in conversation_traits:
    #     text += f"<b>{trait}</b>: {conversation_traits[trait]}<br><br>"
    starter = f"""
  {{
    "Name": "Target Personality: {cha}",
    "TestID": "{cha}",
    "Text": "{text}",
    "Files": {{\n"""
    modelpaths = """"""
    for trait in conversation_traits:
        added = f'      "<b>{trait}: </b>{conversation_traits[trait]}": "./audio/sample.wav",\n'
        modelpaths += added
    ##########################
    modelpaths += f'    }}\n'
    modelpaths += f'  }},'
    whole_text += starter+modelpaths
whole_text += "\n]\n}}"
print(whole_text[:-1])
savefile = f"./config/expected_trend.js"
try:
    os.remove(savefile)
except FileNotFoundError:
    pass
if save:
    f = open(savefile, "a")
    f.write(whole_text[:-1])
    f.close()

In [ ]:
save_dir = "/Users/shoahoshoaho/Downloads/samples/"
exid = "hindi"
num = 20

dirnames = experiments[exid]
dirname = dirnames[0]
dirname = f'{"___".join(dirname.split("/"))}'
filenames = [os.path.basename(a)[:-4] for a in glob.glob(result_base_dir+f"{dirname}/*.wav")]
filenames.sort()
np.random.seed(42)
bl = np.random.choice(np.arange(len(filenames)), size=num, replace=False)
for b, basename in enumerate(np.array(filenames)[bl]):
    print(basename)
    spk, name = basename.split("_")
    gt_path = result_base_dir + f"ground_truth/{basename}.wav"
    for d, dirname in enumerate(dirnames):
        print(dirname)
        savepath = save_dir + f"{b}_{d}.wav"
        dirname = f'{"___".join(dirname.split("/"))}'
        modelpath = result_base_dir+f"{dirname}/{basename}.wav"
        play_audio(modelpath, 16000)
        shutil.copy(modelpath, savepath)